# Twitter Miner Notebook with MongoDB Integration

### Setting environment variables

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# OAuth keys
CONSUMER_KEY = os.environ.get('CONSUMER_KEY')
CONSUMER_SECRET = os.environ.get('CONSUMER_SECRET')
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.environ.get('ACCESS_TOKEN_SECRET')

# MongoDB values
MONGO_PORT = os.getenv('MONGO_PORT', 27017)
MONGO_URL = os.getenv('MONGO_URL', 'localhost')

### Setting MongoDB Connection

In [ ]:
import json
from pymongo import MongoClient
import pymongo

client = MongoClient(MONGO_URL, MONGO_PORT)
db = client.twitterdb
col = db.tweets
raw_col = db.raw_data
keywords = ['bieber']

### Defining a timer function for the thread

In [ ]:
import threading
import sys
import os
import time

def count_time( threadName, delay, limit):
    count = 0
    while count < limit:
        time.sleep(delay)
        count += 1
        print ("%s: %s" % ( threadName, time.ctime(time.time()) )  )
    print('finished ', threadName)
    #os._exit(0)

### Creating class for the Streaming

In [ ]:
class TwitterListener(StreamListener):
    def __init__(self):
        super().__init__()
        self.counter = 0
        self.limit = 100
        
        #using multithread on the object construction
        #threading.Thread(target = count_time,
            #args= ("Time Counter", 1, 5)
        #).start()
        
    def on_data(self,data):
        
        tweet = json.loads(data)

        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        text = tweet["text"]
        
        user_id = tweet["user"]["id_str"]
        user_name = tweet["user"]["name"]
        screen_name = tweet["user"]["screen_name"]
        user_created_at = tweet["user"]["created_at"]
        
        obj = {
            "message_created_at":time.mktime(time.strptime( created_at,"%a %b %d %H:%M:%S +0000 %Y" )),
            "message_id":id_str,
            "message_text":text,
            "author_user_id": user_id,
            "author_created_at": time.mktime(time.strptime( user_created_at,"%a %b %d %H:%M:%S +0000 %Y" )),
            "author_user_name": user_name,
            "author_screen_name": screen_name
        }
        
        #print('🐦', end='')

        tweetind = col.insert_one(obj).inserted_id

        tweetind_2 = raw_col.insert_one( json.loads(json.dumps(tweet)) ).inserted_id
        
        #Tweet limitation counter
        self.counter += 1
        if self.counter < self.limit:
            return True
        else:
            return False
        
        return True
    
    def on_error(self, status):
        print('error code: ', status)
        return False

### OAuth Implementation

In [ ]:
from tweepy.streaming import StreamListener
from tweepy.auth import OAuthHandler
from tweepy import Stream

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

### Initializing Stream

In [ ]:
twitter_listener = TwitterListener()
twitter_stream = Stream(auth, listener = twitter_listener)

In [ ]:
print('Start stream')
#mystream.filter(track=keywords)
twitter_stream.filter(track = keywords, is_async = True)
#mystream.filter(track = keywords, async = True)

time.sleep(30)

twitter_stream.disconnect()

print('Close stream')

### Saving to from MongoDB to .tsv file.

In [ ]:
import json
import csv
from bson.json_util import dumps

json_data = json.loads(dumps( 
    col.find().sort(
        [("message_created_at",pymongo.ASCENDING),
         ("author_created_at",pymongo.ASCENDING)]) 
))

with open('output.tsv', 'w') as output_file:
    dw = csv.DictWriter(
        output_file,
        sorted( json_data[0].keys() ),
        delimiter='\t')
    
    dw.writeheader()
    dw.writerows(json_values)